## **Required Packages  :**

In [2]:
import datetime
import os, sys, shutil
import numpy as np
from numpy import loadtxt
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import optimizers, applications
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras import models
from tensorflow.keras.models import Model,load_model
from tensorflow.keras import optimizers,regularizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import TensorBoard , ModelCheckpoint
from tensorflow_model_optimization.sparsity import keras as sparsity
import zipfile
import time
import tempfile
import seaborn as sns

In [3]:
bacth_size = 16
epochs = 30
warmup_epocks = 2
learning_rate = 0.0005
warmup_learning_rate = 0.0001
height = 128
width = 128
colors = 3
n_classes = 5
es_patience = 18
rlrop_patience = 5
decay_drop = 0.5
based_model_last_block_layer_number = 5 

In [4]:
train_dir = 'C:/Users/asus/Desktop/images/train'
validation_dir = 'C:/Users/asus/Desktop/images/validation'

## **Data Augmentation :** 

In [5]:

train_datagen = ImageDataGenerator(
      rescale=1/255,
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.5,
      brightness_range=[0.7,1.3],
      horizontal_flip=True,
      fill_mode='nearest')

In [6]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size= bacth_size,
        shuffle = True,
        class_mode= 'categorical')


Found 2716 images belonging to 5 classes.


In [7]:
val_datagen = ImageDataGenerator(rescale=1/255)

val_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(height, width),
        batch_size = bacth_size,
        shuffle=True,
        class_mode= 'categorical')

Found 941 images belonging to 5 classes.


In [8]:
from keras.layers import Flatten,GlobalMaxPooling2D
from keras import regularizers
from tensorflow.keras.layers import BatchNormalization

Using TensorFlow backend.


## **Model Creation : **

In [9]:
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = applications.Xception(weights='imagenet', #à modifier
                                       include_top=False,
                                       input_tensor=input_tensor)
    x= GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.4)(x)
    output = Dense(1028, activation='elu',kernel_regularizer=regularizers.l2(0.001), name='output')(x)
    output = Dropout(0.5)(output)
    model_prim = Model(input_tensor, output)
    final_output = Dense(n_out, activation='softmax',kernel_regularizer=regularizers.l2(0.001), name='final_output')(model_prim.output)
    model = Model(input_tensor, final_output)

    return model
model = create_model(input_shape=(height, width, colors), n_out=n_classes)


In [10]:

for layer in model.layers[:based_model_last_block_layer_number]:
    layer.trainable = False
for layer in model.layers[based_model_last_block_layer_number:]:
    layer.trainable = True


metric_list = ["accuracy"]
optimizer = optimizers.Adam(lr=warmup_learning_rate)
model.compile(optimizer=optimizer, loss="categorical_crossentropy",  metrics=metric_list)
model.summary()


# Early Stopping :
es = EarlyStopping(monitor='val_loss', mode='min', patience=es_patience, verbose=1)
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=rlrop_patience, factor=decay_drop, min_lr=1e-6, verbose=1)


step_train = train_generator.n//train_generator.batch_size
step_validation = val_generator.n//val_generator.batch_size

print(train_generator.n)

            
__________________________________________________________________________________________________
block5_sepconv3_act (Activation (None, 8, 8, 728)    0           block5_sepconv2_bn[0][0]         
__________________________________________________________________________________________________
block5_sepconv3 (SeparableConv2 (None, 8, 8, 728)    536536      block5_sepconv3_act[0][0]        
__________________________________________________________________________________________________
block5_sepconv3_bn (BatchNormal (None, 8, 8, 728)    2912        block5_sepconv3[0][0]            
__________________________________________________________________________________________________
add_3 (Add)                     (None, 8, 8, 728)    0           block5_sepconv3_bn[0][0]         
                                                                 add_2[0][0]                      
________________________________________________________________________________________________

In [12]:
logdir = tempfile.mkdtemp()
print('Writing training logs to ' + logdir)
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir, profile_batch=0)]



Writing training logs to C:\Users\asus\AppData\Local\Temp\tmpj_e7ydcx


In [ ]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=step_train,
                              epochs=30,
                              verbose=1 ,
                              callbacks=callbacks,                            
                              validation_data=val_generator,
                              validation_steps=step_validation

                            ).history

In [15]:
model.save('Xception_images.h5')

train_data = pd.read_csv('C:/Users/asus/Desktop/retino/train.csv')
test_data = pd.read_csv('C:/Users/asus/Desktop/retino/test.csv')

train_path = 'C:/Users/asus/Desktop/retino/images/train'
test_path = 'C:/Users/asus/Desktop/retino/images/test'


## **Image Preprocessing:**

In [ ]:
def preprocess_image(image_path, desired_size=128):
    im = Image.open(image_path)
    im = im.resize((desired_size,) * 2, resample=Image.LANCZOS)

    return im

N = train_data.shape[0]

x_train = np.empty((N, 128, 128, 3), dtype=np.uint8)


for i, image_id in enumerate(tqdm(train_data['id_code'])):
    x_train[i, :, :, :] = preprocess_image(
        os.path.join(train_path + "/" + image_id + '.png')
    )

x_train = x_train / 255

## **Predictions :**

In [16]:

# use the model to generate predictions for all of the training images
start = datetime.datetime.now()
print('Started predicting at {}'.format(start))

train_prediction = model.predict([x_train])

end = datetime.datetime.now()
elapsed = end - start
print('Predicting took a total of {}'.format(elapsed))

# take the highest predicted probability for each image
train_predictions = [np.argmax(pred) for pred in train_prediction]

100%|██████████| 3662/3662 [15:59<00:00,  3.82it/s]
Started predicting at 2020-04-22 11:19:34.413887
Predicting took a total of 0:01:25.824835


## **The model Performance :**

In [ ]:
# look at how the model performed for each class
labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
cnf_matrix = confusion_matrix(train_data['diagnosis'].astype('int'), train_predictions)
cnf_matrix_norm = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
df_cm = pd.DataFrame(cnf_matrix_norm, index=labels, columns=labels)
plt.figure(figsize=(16, 7))
sns.heatmap(df_cm, annot=True, fmt='.2f', cmap="Blues")
plt.show()


plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()



## **Pruning :** 

In [17]:
# Backend agnostic way to save/restore models
_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Saving model to:  C:\Users\asus\AppData\Local\Temp\tmphya3uqs9.h5


In [19]:
epochs = 10
num_train_samples = x_train.shape[0]
end_step = np.ceil(1.0 * num_train_samples / train_generator.batch_size).astype(np.int32) * epochs
print('End step: ' + str(end_step))

End step: 2290


In [20]:
pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=0,
                                                   end_step=end_step)
}

### **Pruned Model :**

In [ ]:

l = tf.keras.layers

pruned_model = tf.keras.Sequential()
    
for layer in model.layers:

    if (layer.name == 'output') or (layer.name == 'final_output') or (layer.name == 'block_conv1') or (layer.name == 'block_conv2') or (layer.name == 'conv2d_1'):
        pruned_model.add(sparsity.prune_low_magnitude(layer,input_shape=(height, width, colors),**pruning_params))
    else:
        try:
            pruned_model.add(layer)
        except ValueError:
               pass

model = pruned_model

In [ ]:
logdir = tempfile.mkdtemp()
print('Writing training logs to ' + logdir)

In [ ]:
pruned_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])


In [ ]:

# Add a pruning step callback to peg the pruning step to the optimizer's
# step. Also add a callback to add pruning summaries to tensorboard
callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

history_1 = pruned_model.fit(train_generator,
          steps_per_epoch=step_train,
          epochs=5,
          verbose=1,
          callbacks=callbacks,
          validation_data=val_generator).history
          

In [ ]:
pruned_model.summary()

In [ ]:
_, checkpoint_file = tempfile.mkstemp('.h5')
print('Saving pruned model to: ', checkpoint_file)
# saved_model() sets include_optimizer to True by default. Spelling it out here
# to highlight.
tf.keras.models.save_model(pruned_model, checkpoint_file, include_optimizer=True)

with sparsity.prune_scope():
  restored_model = tf.keras.models.load_model(checkpoint_file)

callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

history_2 = restored_model.fit(train_generator,
                    steps_per_epoch=step_train,
                   epochs=5,
                   verbose=1,
                   callbacks=callbacks,
                   validation_data=val_generator).history


In [ ]:
restored_model.summary()

In [ ]:
final_model = sparsity.strip_pruning(pruned_model)
final_model.summary()

In [ ]:

# use the model to generate predictions for all of the training images
start = datetime.datetime.now()
print('Started predicting at {}'.format(start))

train_prediction = final_model.predict([x_train])

end = datetime.datetime.now()
elapsed = end - start
print('Predicting took a total of {}'.format(elapsed))

# take the highest predicted probability for each image
train_predictions = [np.argmax(pred) for pred in train_prediction]

# look at how the model performed for each class
labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
cnf_matrix = confusion_matrix(train_data['diagnosis'].astype('int'), train_predictions)
cnf_matrix_norm = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
df_cm = pd.DataFrame(cnf_matrix_norm, index=labels, columns=labels)
plt.figure(figsize=(16, 7))
sns.heatmap(df_cm, annot=True, fmt='.2f', cmap="Blues")
plt.show()
import matplotlib.pyplot as plt

plt.plot(history_2['accuracy'])
plt.plot(history_2['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_2['loss'])
plt.plot(history_2['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
_, pruned_keras_file = tempfile.mkstemp('.h5')
print('Saving pruned model to: ', pruned_keras_file)

# No need to save the optimizer with the graph for serving.
tf.keras.models.save_model(final_model, pruned_keras_file, include_optimizer=False)

### **Comparison of  size of the  pruned /unpruned model before and after compression:**

In [ ]:
_, zip1 = tempfile.mkstemp('.zip') 
with zipfile.ZipFile(zip1, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(keras_file)
print("Size of the unpruned model before compression: %.2f Mb" % 
      (os.path.getsize(keras_file) / float(2**20)))
print("Size of the unpruned model after compression: %.2f Mb" % 
      (os.path.getsize(zip1) / float(2**20)))

_, zip2 = tempfile.mkstemp('.zip') 
with zipfile.ZipFile(zip2, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(pruned_keras_file)
print("Size of the pruned model before compression: %.2f Mb" % 
      (os.path.getsize(pruned_keras_file) / float(2**20)))
print("Size of the pruned model after compression: %.2f Mb" % 
      (os.path.getsize(zip2) / float(2**20)))

